In [1]:
import pandas as pd
import numpy as np
import math
import itertools

In [2]:
def impute (data, categorical):
    class_dfs = []
    classes = np.unique(data['target'])
    #goes through each label to calculate the conditional mean
    for label in classes:
        #change the "?" to nan so we can compute the mean skipping the nans
        data_with_label = data[data['target']==label].replace('?', np.nan).astype('float')
        means = data_with_label.mean(skipna=True)
        #goes through each feature to replace empty vals with conditional means
        #rounds the mean if the data is categorical
        for feature in data.columns:
            if categorical:
                replacement_val = round(means[feature])
            else:
                replacement_val = means[feature]
            data_with_label[feature] = data_with_label[feature].apply(lambda x: replacement_val if math.isnan(x) else x)
        class_dfs.append(data_with_label)
    return pd.concat(class_dfs).reset_index().drop(columns='index')
#split the data into 5 different sets and return each set
def validation_sets(data):
    sets = []
    sample_size = len(data)
    class_splits = []
    for class_val in np.unique(data['target']):
        df_class = data[data['target'] == class_val]
        dfs_class = np.array_split(df_class, 5)
        class_splits.append(dfs_class)
    for index in range(0, 5):
        sample_set_list = []
        for class_index in range(len(class_splits)):
            sample_set_list.append(class_splits[class_index][index])
        sample_set = pd.concat(sample_set_list)
        sets.append(sample_set.reset_index().drop(columns= 'index'))
    return sets

#this is the shell for all our models. it takes each set and tests it against the rest of the sets and returns the accuracy
#to evaluate our model. it also allows for  different preprocessing, layer values/sizes, learning rates, epochs and momentum
def k_fold_cross_validation(sets, preprocess, layers, max_hidden_layers, outputs, learning_rate, epochs, momentum):
    #the below creates a list of all the possible permutations of the hidden units for each layer for each number of layers
    layers_list = []
    for layer_number in range(max_hidden_layers):
        layer_list = list(itertools.combinations(layers, layer_number+1))
        layer_list = [list(vals)+[outputs] for vals in layer_list]
        layers_list+=layer_list
    hyperparamaters = list(itertools.product(*[layers_list, learning_rate, epochs, momentum]))
    hyperparamaters = [list(vals) for vals in hyperparamaters]
    scores = [[] for i in range(len(hyperparamaters))]
    for index in range(0, len(sets)):
        test_set = sets[index]
        #concats the rest of the sets for training
        training_set = pd.concat([t_set for (set_index, t_set) in enumerate(sets) if set_index!=index])
        labels = len(np.unique(training_set['target']))
        y= test_set['target']
        x_train = training_set.reset_index().drop(columns= ['index','target']).values
        x_test = test_set.reset_index().drop(columns= ['index','target']).values
        #one hot encode the labels if this is a multiclass problem
        if labels>2:
            y_train= (np.eye(labels)[training_set['target']])
            y_test= (np.eye(labels)[test_set['target']])
        else:
            y_train = np.array(training_set['target']).reshape(-1,1)
            y_test = np.array(test_set['target']).reshape(-1,1)
        for index, hyperparamater in enumerate(hyperparamaters):
            hyperparamater = list(hyperparamater)
#this was my original approach to finding the optimal number of hidden layers and units. I give detail on this in my paper
#thought it might be interesting to leave this here commented out
#             if dynamic_nodes:
#                 hidden_layers = len(hyperparamater[0])-1
#                 current_layers = [1]*hidden_layers + [hyperparamater[0][-1]]
#                 best_layers = current_layers
#                 mlp = MLP(preprocess=preprocess, layers = current_layers, learning_rate = hyperparamater[1], epochs = hyperparamater[2], momentum = hyperparamater[3])
#                 mlp.fit(x_train, y_train)
#                 y_predict = mlp.predict(x_test)
#                 results = [y_predict[result_index]==actual for result_index, actual in enumerate(y)]
#                 best_score = results.count(True)/len(results)
                
#                 for layer_index in range(hidden_layers):
#                     decrease = 0
#                     current_layers = best_layers
#                     while decrease <=5:
#                         current_layers[layer_index]+=1
#                         print(current_layers)
#                         mlp = MLP(preprocess=preprocess, layers = current_layers, learning_rate = hyperparamater[1], epochs = hyperparamater[2], momentum = hyperparamater[3])
#                         mlp.fit(x_train, y_train)
#                         y_predict = mlp.predict(x_test)
#                         results = [y_predict[result_index]==actual for result_index, actual in enumerate(y)]
#                         current_score = results.count(True)/len(results)
#                         print(current_score)
#                         if current_score>best_score:
#                             best_layers = current_layers
#                             best_score = current_score
#                             decrease = 0
#                         else:
#                             decrease+=1
#                 print(best_layers)
#                 hyperparamater[0] = best_layers
#                 hyperparamaters[index][0] = best_layers
            mlp = MLP(preprocess=preprocess, layers = hyperparamater[0], learning_rate = hyperparamater[1], epochs = hyperparamater[2], momentum = hyperparamater[3])
            mlp.fit(x_train, y_train)
            y_predict = mlp.predict(x_test)
            results = [y_predict[result_index]==actual for result_index, actual in enumerate(y)]
            scores[index].append(results.count(True)/len(results))
    print('MLP accuracy:')
    for index, hyperparamaters in enumerate(hyperparamaters):
        print(f'layers = {hyperparamaters[0]} learning_rate = {hyperparamaters[1]} epochs = {hyperparamaters[2]} momentum = {hyperparamaters[3]}: {np.mean(scores[index])}')
        

In [3]:
#this is our class for a multilayer perceptron
class MLP():
    #init function for initializing the variables we will need across epochs for training and prediction 
    def __init__(self, preprocess, layers, learning_rate, epochs, momentum=0):
        self.layers = layers
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.momentum = momentum
        self.weights = []
        self.weight_updates = [[] for _ in range(len(layers))]
        self.prior_weight_updates = [[] for _ in range(len(layers))]
        self.bias_updates = [[] for _ in range(len(layers))]
        self.prior_bias_updates = [[] for _ in range(len(layers))]
        self.biases = []
        self.outputs = [[] for _ in range(len(layers))]
        self.inputs = [[] for _ in range(len(layers))]
        self.units = [[] for _ in range(len(layers))]
        self.lr_preprocess = preprocess
        
    #here the weights are initialized to a random number between [-0.01,0.01] the size of the weight matrices correspond
    #to the size of the previous layer and the size of the next layer: shape = (previous, next)
    def initialize_weights (self, feature_count):
        previous = None
        for index, layer in enumerate(self.layers):
            if previous:
                initialized_weight = np.random.uniform(low=-0.01, high=0.01, size=(previous, layer))
                initialized_bias = np.random.uniform(low=-0.01, high=0.01, size=layer)
            else:
                initialized_weight = np.random.uniform(low=-0.01, high=0.01, size=(feature_count, layer))
                initialized_bias = np.random.uniform(low=-0.01, high=0.01, size=layer)
            previous = layer
            self.weights.append(initialized_weight)
            self.biases.append(initialized_bias)
    #the sigmoid function is used to compute the hidden units and the output values for binary classification
    def sigmoid(self, linear_function):
        return 1/(1+ np.exp(-linear_function))
    #the softmax function is used to compute the output values for multiclass problems
    def softmax(self, linear_function):
        return np.exp(linear_function) / np.sum(np.exp(linear_function),axis=1,keepdims=True)
    #the derivative of the sigmoid function is a required variable for the weight updates of the hidden layers
    def sigmoid_deriv(self, linear_function):
        sigmoid_val = self.sigmoid(linear_function)
        return sigmoid_val*(1-sigmoid_val)
    #this is not used but it is the derivative of the softmax function
    def softmax_deriv(self, x):
        return x*(1-x)
    #this function forward propogates the samples through the network given the current weights and computes the output
    #the output is computed using sigmoid for binary and softmax for multiclass
    def forward_prop(self, x):
        for index in range(len(self.layers)):
            #first hidden layer
            if index ==0:
                self.units[index] = np.array(x)
                self.inputs[index] = np.dot(x, self.weights[index]) + self.biases[index]
                self.outputs[index] = self.sigmoid(self.inputs[index])
            #output layer
            elif index == len(self.layers)-1:
                self.units[index] = np.array(self.outputs[index-1])
                self.inputs[index] = np.dot(self.outputs[index-1], self.weights[index]) + self.biases[index]
                if self.layers[index]>1:
                    self.outputs[index] = self.softmax(self.inputs[index])
                else:
                    self.outputs[index] = self.sigmoid(self.inputs[index])
            #additional hidden layers
            else:
                self.units[index] = self.outputs[index-1]
                self.inputs[index] = np.dot(self.outputs[index-1], self.weights[index]) + self.biases[index]
                self.outputs[index] = self.sigmoid(self.inputs[index])
    #this function backpropogates 
    def back_prop(self, x, y):
        y_pred = np.array(self.outputs[-1])
        diff = (y_pred - y)
        for index in range(len(self.layers)-1, -1, -1):
            #output layer
            if index == len(self.layers)-1:
                error = diff
                self.prior_weight_updates[index] = self.weight_updates[index]
                self.prior_bias_updates[index] = self.bias_updates[index]
                self.weight_updates[index] = self.outputs[index-1].T.dot((error))
                self.bias_updates[index] = np.sum(error, axis=0)
            #additional hidden layers
            elif(index!=0):
                error = np.multiply((error).dot(self.weights[index+1].T),self.sigmoid_deriv(self.inputs[index]))
                self.prior_weight_updates[index] = self.weight_updates[index]
                self.prior_bias_updates[index] = self.bias_updates[index]
                self.weight_updates[index] = np.dot(self.units[index].T, error)
                self.bias_updates[index] = np.sum(error, axis=0)
            #first hidden layer
            else:
                self.prior_weight_updates[index] = self.weight_updates[index]
                self.prior_bias_updates[index] = self.bias_updates[index]
                self.weight_updates[index] = np.dot(self.units[index].T, np.multiply((error).dot(self.weights[index+1].T),self.sigmoid_deriv(self.inputs[index])))
                self.bias_updates[index] = np.sum((error).dot(self.weights[index+1].T)*self.sigmoid_deriv(self.inputs[index]), axis=0)
        #updating weights allowing for momentum if specified
        for index in range(len(self.weights)):
            weight_update = self.weight_updates[index]*self.learning_rate
            bias_update = self.bias_updates[index]*self.learning_rate
            if self.momentum > 0 and len(self.prior_weight_updates[index])!=0:
                weight_update+=self.prior_weight_updates[index]*self.momentum*self.learning_rate
                bias_update+=self.prior_bias_updates[index]*self.momentum*self.learning_rate
            self.weights[index] -= weight_update
            self.biases[index] -= bias_update
    #training function utilizing a batch learning approach, also allows for preprocessing of features
    #training involved forward prop and back prop for the specified number of epochs
    def fit(self, x, y):
        x= self.preprocess_features(x)
        self.initialize_weights(x.shape[1])
        for epoch in range(self.epochs):
            n_samples = x.shape[0]
            try:
                batch_args = np.random.choice(n_samples, 50, replace = False)
            except Exception:
                batch_args = np.random.choice(n_samples, 32, replace = False)
            x_batch = x[batch_args]
            y_batch = y[batch_args]
            self.forward_prop(x_batch)
            self.back_prop(x_batch,y_batch)
    #different types of preprocessing for the features, default = none
    def preprocess_features(self, x):
        if self.lr_preprocess == 'standardize':
            return (x-np.mean(x,axis=0))/np.std(x,axis= 0)
        elif self.lr_preprocess == 'normalize':
            return (((x-np.min(x, axis=0))/(np.max(x, axis=0)-np.min(x, axis=0)))*-1)+1
        else:
            return x
    #this is the predict method for binary classification, it uses the sigmoid approach
    def predict_binary(self, x):
        output= []
        output.append(x)
        for index in range(len(self.layers)):
            if index == len(self.layers)-1:
                output.append(self.sigmoid(np.dot(output[index], self.weights[index]) + self.biases[index]))
            else:
                output.append(self.sigmoid(np.dot(output[index], self.weights[index]) + self.biases[index]))
        out = [1 if x>=0.5 else 0 for x in output[-1]]
        return out
    #this is the predict method for multiclass classification, it uses the softmax approach
    def predict_multiclass(self, x):
        output= []
        output.append(x)
        for index in range(len(self.layers)):
            if index == len(self.layers)-1:
                output.append(self.softmax(np.dot(output[index], self.weights[index]) + self.biases[index]))
            else:
                output.append(self.sigmoid(np.dot(output[index], self.weights[index]) + self.biases[index]))
        out = [np.argmax(x) for x in output[-1]]
        return out
    #this is the wrapper method for predictions, it checks if the problem is binary or multiclass to choose the right prediction approach
    def predict(self, x):
        x= self.preprocess_features(x)
        x=np.array(x)
        if (self.layers[-1]>1):
            out = self.predict_multiclass(x)
        else:
            out = self.predict_binary(x)
        return out
        

In [4]:
#importing iris data
iris = pd.read_csv('iris.data', sep=",", header=None)
iris.columns = ["sepal length in cm", "sepal width in cm", "petal length in cm", "petal width in cm", "target"]

In [5]:
iris['target'] = iris['target'].replace('Iris-setosa', 0).replace('Iris-versicolor', 1).replace('Iris-virginica', 2)
#labels = len(np.unique(iris['target']))
#y = (np.eye(labels)[iris['target']])
#x = iris.drop(columns = 'target')

In [6]:
sets = validation_sets(iris)
k_fold_cross_validation(sets, preprocess = None, layers= [30, 8], max_hidden_layers = 2, outputs = 3, learning_rate=[0.01], epochs = [500], momentum = [0.6])


MLP accuracy:
layers = [30, 3] learning_rate = 0.01 epochs = 500 momentum = 0.6: 0.9800000000000001
layers = [8, 3] learning_rate = 0.01 epochs = 500 momentum = 0.6: 0.9866666666666667
layers = [30, 8, 3] learning_rate = 0.01 epochs = 500 momentum = 0.6: 0.8933333333333333


In [7]:
#importing breast cancer data and removing the id column
#"Missing Attribute Values: Denoted by "?"" so we need to impute those
breast_cancer = pd.read_csv('breast-cancer-wisconsin.data', sep=",", header=None)
breast_cancer.columns = ["id", "Clump Thickness", "Uniformity of Cell Size", "Uniformity of Cell Shape", "Marginal Adhesion",
               "Single Epithelial Cell Size", "Bare Nuclei", "Bland Chromatin", "Normal Nucleoli", "Mitoses", "target"]
breast_cancer = breast_cancer.drop(columns='id')

In [8]:
#our data labels 2 for benign and 4 for malignant, so let's relabel this to 0 and 1 respectively
breast_cancer['target'] = breast_cancer['target'].replace(2, 0).replace(4, 1)

#the missing attributes are numerical, so we impute the data by 
#finding the mean for that meature given the class
breast_cancer = impute(breast_cancer, False)

In [9]:
sets = validation_sets(breast_cancer)
k_fold_cross_validation(sets, preprocess = 'standardize', layers= [9], max_hidden_layers = 2, outputs=1, learning_rate=[0.01], epochs = [250], momentum=[0])

MLP accuracy:
layers = [9, 1] learning_rate = 0.01 epochs = 250 momentum = 0: 0.9685808313835255


In [10]:
#importing house vote data
#Missing Attribute Values: Denoted by "?"
house_vote = pd.read_csv('house-votes-84.data', sep=",", header=None)
house_vote.columns = ["target", "handicapped-infants", "water-project-cost-sharing", "adoption-of-the-budget-resolution", "physician-fee-freeze",
               "el-salvador-aid", "religious-groups-in-schools", "anti-satellite-test-ban", "aid-to-nicaraguan-contras", "mx-missile", "immigration",
               "synfuels-corporation-cutback", "education-spending", "superfund-right-to-sue", "crime", "duty-free-exports", "export-administration-act-south-africa"]

In [11]:
#our labels are democrat and republican, so let's relabel them to 0 and 1 respectively
house_vote['target'] = house_vote['target'].replace('democrat', 0).replace('republican', 1)

#labels are represented as "y" or "n", so we change this to "1" and "0" respectively
house_vote = house_vote.replace("y", 1).replace("n", 0)
#we also have some "?" for missing data. Given that this data is categorical, let's take the mean given the class and round it to 0 or 1 for imputation
house_vote = impute(house_vote, True)

In [12]:
sets = validation_sets(house_vote)

k_fold_cross_validation(sets, preprocess = None, layers= [3], max_hidden_layers = 2, outputs=1, learning_rate=[0.01], epochs = [750], momentum=[0])


MLP accuracy:
layers = [3, 1] learning_rate = 0.01 epochs = 750 momentum = 0: 0.9700323199922238


In [13]:
glass = pd.read_csv('glass.data', sep=",", header=None)
glass.columns = ['id', 'ri', 'na', 'mg', 'al', 'si', 'k', 'ca', 'ba', 'fe', 'target']
glass = glass.drop(columns = 'id')
glass['target'] = glass['target'].replace(7, 0).replace(6, 4)

In [14]:
sets = validation_sets(glass)
k_fold_cross_validation(sets, preprocess = 'standardize', layers= [30], max_hidden_layers = 3, outputs=6, learning_rate=[0.01], epochs = [500], momentum = [0.75])


MLP accuracy:
layers = [30, 6] learning_rate = 0.01 epochs = 500 momentum = 0.75: 0.5830690627202254


In [15]:
soybean = pd.read_csv('soybean-small.data', sep=",", header=None)
soybean.columns = ['date','plant-stand','precip','temp','hail','crop-hist','area-damaged','severity','seed-tmt','germination','plant-growth','leaves','leafspots-halo','leafspots-marg','leafspot-size','leaf-shread','leaf-malf','leaf-mild','stem','lodging','stem-cankers','canker-lesion','fruiting-bodies','external decay',
'mycelium','int-discolor','sclerotia','fruit-pods','fruit spots','seed','mold-growth','seed-discolor','seed-size','shriveling','roots', 'target']
soybean['target'] = soybean['target'].replace('D1', 0).replace('D2', 1).replace('D3', 2).replace('D4', 3)
soybean = impute(soybean, True)
soybean_new = pd.get_dummies(soybean.drop(columns = 'target'), columns =  ['date','plant-stand','precip','temp','hail','crop-hist','area-damaged','severity','seed-tmt','germination','plant-growth','leaves','leafspots-halo','leafspots-marg','leafspot-size','leaf-shread','leaf-malf','leaf-mild','stem','lodging','stem-cankers','canker-lesion','fruiting-bodies','external decay',
'mycelium','int-discolor','sclerotia','fruit-pods','fruit spots','seed','mold-growth','seed-discolor','seed-size','shriveling','roots'])
soybean_new['target'] = np.array(soybean['target'], dtype=int)


In [16]:
sets = validation_sets(soybean_new)
k_fold_cross_validation(sets, preprocess = None, layers= [15],max_hidden_layers=1, outputs = 4, learning_rate=[0.01], epochs = [250], momentum = [0])


MLP accuracy:
layers = [15, 4] learning_rate = 0.01 epochs = 250 momentum = 0: 1.0
